We want to test the steady states obtained form the optimal $\Gamma^{(L)}$ outputted by CVX. This code lets us do that. First we need to import our modules and import our data.

In [109]:
from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io


matlab_data = scipy.io.loadmat('../matlab/thermal_data.mat',mat_dtype=False)  # is set to True, givex complex casting to real errors..



In [110]:
beta = matlab_data["beta"][0,0]
beta = float(beta)
print('beta is ' ,beta)

deltalist = matlab_data["deltalist"][:,0]
deltalist = deltalist.astype('float')
print('deltalist is ', deltalist)

w0list = matlab_data["w0list"][:,0]
w0list = w0list.astype('float')
print('w0list is ', w0list)

glist = matlab_data["glist"][:,0]
glist = glist.astype('float')

print('glist is', glist)

NL = int(matlab_data["NL"][0,0])
NM = int(matlab_data["NM"][0,0])

print('NL and NM are ', NL, NM, 'respectively')

N = NL + NM
dL = 2**NL
dM = 2**NM
d = 2**N
dims = [2]*N

# Parameters can be modified here for testing purposes. 



beta is  1.0
deltalist is  [1. 1. 1. 1. 1.]
w0list is  [1. 1. 1. 1. 1. 1.]
glist is [0.1 0.1 0.1 0.1 0.1]
NL and NM are  2 4 respectively


In [111]:
H_S = create_hamiltonian_v2(w0list,glist,deltalist,N)
eigenenergies, eigenstates = H_S.eigenstates()
number = len(eigenenergies) # should be 2^N




In [112]:
matlab_F_list = matlab_data["F"]
F_list = []


for index in range(2**dL):
    matrix = np.asmatrix(matlab_F_list[0,index])

    F_list.append(Qobj(matrix))
    
if not basis_is_orthonormal(F_list):
    print("WARNING : Basis is NOT orthonormal")

F_list.pop() #removes the last indentity basis ..

for index in range(2**dL-1):
    F_list[index].dims = [dims,dims] # makes sure mutliplication will work and QuTiP wont throw errors. 

gamma_matrix = np.asmatrix(matlab_data["gamma_matrix"])




In [113]:
[D,U ] = np.linalg.eigh(gamma_matrix)

Note that we have gotten now $ U^\dagger \Gamma U = D$ where $U$ is a diagonal matrix. Therefore the Dissipator term is written as
$$
D(\rho) = \sum_{\alpha, \beta}  \Gamma_{\alpha,\beta} (F_\beta \rho F_\alpha^\dagger - \{ F_\alpha^\dagger F_\beta, \rho\}/2 ) \\
= \sum_{\alpha, \beta} \sum_{i,j} (U_{\alpha,i} D_{i,j} U^\dagger_{j,\beta})  (F_\beta \rho F_\alpha^\dagger - \{ F_\alpha^\dagger F_\beta, \rho\}/2 )  \\
 = \sum_{i,j}  D_{i,j}   (L_j \rho L_i^\dagger - \{ L_i^\dagger L_j, \rho\}/2 ) $$

where $L_j = \sum_{\beta} U^\dagger_{j,\beta} F_\beta $
We can define $\tilde{L}_j = \sqrt{D_{jj}} L_j$ to get
$$ D(\rho) = \sum_j ( \tilde{L}_j \rho \tilde{L}_j^\dagger - \{ \tilde{L}_j^\dagger \tilde{L}_j, \rho\}/2 ) 

In [114]:
## Constructing the lindblad operators we have

L_list = []
Udagger = np.transpose(np.conj(U))

for j  in range(dL**2-1):
    op = 0
    for betaindex in range(dL**2-1):    
        op = op+Udagger[j,betaindex]* F_list[betaindex]
    L_list.append(op)

tildeL_list = []

for index in range(dL**2-1):
    tildeL_list.append(np.sqrt(D[index])*L_list[index])



We are done with constructing Lindblad operators. Now let us compute the L_2 according to this. Then, we will compute the steadystate.


In [115]:
rho_gibbs = (-beta*H_S).expm() 
rho_th = rho_gibbs/rho_gibbs.tr()

rhodot_thermal = 0

for i in range(dL**2-1):
    L = tildeL_list[i]
    rhodot_thermal = rhodot_thermal+ L*rho_th*L.dag() - 0.5*L.dag()*L*rho_th - 0.5*rho_th*L.dag()*L

violation  = np.zeros((d,1))
for i in range(d):
    violation[i] = (eigenstates[i].dag()*rhodot_thermal*eigenstates[i]).diag() #affected by ordering of U


rhodot_thermal_v2 = 0

for  alphaindex in range(dL**2-1):
    for betaindex in range(dL**2-1):
        Falpha = F_list[alphaindex]
       
        Fbeta = F_list[betaindex]
       
        rhodot_therma_v2 = rhodot_thermal_v2 + gamma_matrix[alphaindex,betaindex]*(Fbeta*rho_th*Falpha.dag()-0.5*Falpha.dag()*Fbeta*rho_th - 0.5*rho_th*Falpha.dag()*Fbeta)


violation_v2  = np.zeros((d,1))
for i in range(d):
    violation_v2[i] = (eigenstates[i].dag()*rhodot_thermal_v2*eigenstates[i]).diag() #affected by ordering of U

Now we compute the steady state..

In [116]:
H_LS_matrix = matlab_data["H_LS"]
H_LS = tensor(Qobj(H_LS_matrix),qeye(dM))
H_LS.dims = [dims,dims]
rho_ss = steadystate(H_S+H_LS, tildeL_list)

trace_dist = tracedist(rho_ss,rho_th)


L_operator = liouvillian(H_S+H_LS, tildeL_list)

L_eigen = L_operator.eigenenergies()


We can also compute the SS via a different method by directly constructing L...

In [117]:
L_v2 = spre(-1.0j*(H_S+H_LS)) + spost(1.0j*(H_S+H_LS))

for alphaindex in range(dL**2-1):
    for betaindex in range(dL**2-1):
        Falpha = F_list[alphaindex]
        Fbeta = F_list[betaindex]
        L_v2= L_v2+gamma_matrix[alphaindex,betaindex]*(spre(Fbeta)*spost(Falpha.dag()) - 0.5*spre(Falpha.dag()*Fbeta) - 0.5*spost(Falpha.dag()*Fbeta))


rho_ss_v2 = steadystate(L_v2)
trace_dist_v2 = tracedist(rho_ss_v2,rho_th)
L_eigen_v2 = L_v2.eigenenergies()
        

In [118]:
# print("violation is ",violation)
#print("violation_v2 is ",violation_v2)
# print()
#print("########################################################")
 #print()


print("Trace distance betweens steady state and thermal state is", trace_dist)

print("Smallest eigenvalues of L are ", L_eigen[-3:])

#print()
#print("########################################################")
#print()

print("Trace distance betweens steady state v2 and thermal state is", trace_dist_v2)

print("Smallest eigenvalues of L_v2 are ", L_eigen_v2[-3:])

Trace distance betweens steady state and thermal state is 0.0811477283396092
Smallest eigenvalues of L are  [-2.36069306e-03+1.00011678e+00j -2.36069306e-03-1.00011678e+00j
  4.95111244e-16+3.93144481e-16j]
Trace distance betweens steady state v2 and thermal state is 0.08114772833959769
Smallest eigenvalues of L_v2 are  [-2.36069306e-03+1.00011678e+00j -2.36069306e-03-1.00011678e+00j
 -2.62451595e-15+3.74546141e-15j]


In [119]:
print(eigenenergies) #incase you need to check there are no degenericies

[-3.60000000e+00 -2.60000000e+00 -2.54428304e+00 -2.37767667e+00
 -2.13692895e+00 -1.90671250e+00 -1.63439884e+00 -1.60000000e+00
 -1.54428304e+00 -1.48194392e+00 -1.37767667e+00 -1.34976555e+00
 -1.24153782e+00 -1.13692895e+00 -1.04709831e+00 -9.06712504e-01
 -8.00000000e-01 -6.34398838e-01 -6.00000000e-01 -5.44283041e-01
 -5.38915486e-01 -5.19621977e-01 -4.81943921e-01 -4.10473943e-01
 -3.77676672e-01 -3.49765547e-01 -2.50667203e-01 -2.41537818e-01
 -1.36928946e-01 -4.70983075e-02  8.88178420e-16  2.95502591e-02
  9.32874965e-02  2.00000000e-01  3.16838740e-01  3.65601162e-01
  4.00000000e-01  4.55716959e-01  4.61084514e-01  4.80378023e-01
  5.18056079e-01  6.16516819e-01  6.22323328e-01  6.50234453e-01
  7.49332797e-01  7.58462182e-01  8.63071054e-01  9.52901692e-01
  1.02955026e+00  1.09328750e+00  1.20000000e+00  1.27711838e+00
  1.36560116e+00  1.40000000e+00  1.45571696e+00  1.46108451e+00
  1.48037802e+00  1.62232333e+00  1.74933280e+00  1.86307105e+00
  2.02955026e+00  2.09328